For investors, is a portfolio with higher returns over a period always more desirable than a portfolio with lower returns but a higher sharpe ratio ?

In [6]:
import yfinance as yf
import pandas as pd

ticker = 'NTRS'
start_date = pd.to_datetime('2023-01-01')
end_date = pd.to_datetime('2023-12-31')

data = yf.Ticker(ticker).history(start=start_date, end=end_date)

data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-01-03 00:00:00-05:00,83.468015,84.325051,82.815924,83.635696,719400,0.0,0.0
2023-01-04 00:00:00-05:00,83.971063,86.281341,83.924482,85.405670,1202400,0.0,0.0
2023-01-05 00:00:00-05:00,85.079618,85.685135,84.492729,85.247299,707200,0.0,0.0
2023-01-06 00:00:00-05:00,86.206807,88.470504,84.883989,88.191032,1116900,0.0,0.0
2023-01-09 00:00:00-05:00,88.684773,89.662916,87.650737,87.762527,1043700,0.0,0.0
